In [2]:
############################################################################
#
#  Program: Visualisation de structures VTK, STL ou RT Structure
#
#  Copyright (c) 2016 Jonathan Tranel - IDN-RJ
#  All rights reserved.
#     This software is distributed WITHOUT ANY WARRANTY; without even
#     the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR
#     PURPOSE.  
#
############################################################################

import sys
import gdcm
import dicom
import os
import vtkgdcm
import numpy
import vtk
import itk
sys.path.append('/usr/lib/python27/dist-packages/')
import vtkDICOMPython

from vtk.util.misc import vtkGetDataRoot

#Chemin des DICOM images
PathDicom="/home/jonathan/Bureau/DICOM/UH/"


In [ ]:
###########################################
#Chapitre 1: Statistiques des DICOM images#
###########################################

# Lire les DICOM images      NB: par défaut, critère du image number
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()

# Load dimensions using `GetDataExtent`
ConstPixelDims = reader.GetOutput().GetDimensions()

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

print ("Dimensions [Nb de pixels en x, en y, nombre de coupes]: "+str(ConstPixelDims))

from vtk.util import numpy_support
# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
GrosseMatrice = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
GrosseMatrice = GrosseMatrice.reshape(ConstPixelDims, order='F')

#Inversion   NB: critère de la position z
GrosseMatrice2=GrosseMatrice[:,:,::-1]

print ("Dimensions de GrosseMatrice [Nb de pixels en x, en y, nombre de coupes]: "+str(GrosseMatrice.shape))
#print("Valeur du voxel max pour le fichier "+str(lstFilesDCM[0])+": ")

print ("UH maximal detecte: "+str(GrosseMatrice2[:,:,1].max()))
print ("UH minimal detecte: "+str(GrosseMatrice2[:,:,1].min()))


In [4]:
#####################################################################
#Chapitre 2-1: Importer les RTStruc et DICOM Images et les visualiser#
#####################################################################

#Mettre ici le chemin du fichier RT Structure
CheminRTStructure="/home/jonathan/Bureau/DICOM/RTStruc/RTST0173.dcm"

#Lister toutes les structures du RTStructure
RTStruc=vtkgdcm.vtkGDCMPolyDataReader()
RTStruc.SetFileName(CheminRTStructure)
RTStruc.Update()
Nb=RTStruc.GetRTStructSetProperties().GetNumberOfStructureSetROIs()

for i in range (0,Nb):
    print("Structure n°"+str(i)+": "+RTStruc.GetRTStructSetProperties().GetStructureSetROIName(i))

    
#Mettre ici les RTStruc désirés (après lecture de la liste ci-dessous):

n=8
o=4

Structure n°0: Brainstem 
Structure n°1: Cerebrum
Structure n°2: Chiasm
Structure n°3: Eye, Left 
Structure n°4: Eye, Right
Structure n°5: Lens, Left
Structure n°6: Lens, Right 
Structure n°7: MetFrontG 
Structure n°8: MetOccD1
Structure n°9: MetOccD2
Structure n°10: Optic Nerve, Left 
Structure n°11: Optic Nerve, Right
Structure n°12: Optic Tract, Left 
Structure n°13: Optic Tract, Right


In [5]:
######################################################################
#Chapitre 2-2: Importer les RTStruc et DICOM Images et les visualiser#
######################################################################

#Example stéréo Rangueil

# Lire les DICOM images  NB: par défaut, critère du image number

directory = vtkDICOMPython.vtkDICOMDirectory()
directory.SetDirectoryName(PathDicom)
directory.Update()

readerImg = vtkDICOMPython.vtkDICOMReader()
readerImg.SetFileNames(directory.GetFileNamesForSeries(0))
readerImg.Update()

dim = readerImg.GetOutput().GetDimensions()

#Actor 1: DICOM images par Marching cubes
#Fabrication de la surface 3D du DICOM images
ImgMC = vtk.vtkMarchingCubes()
ImgMC.SetInputConnection(readerImg.GetOutputPort())
ImgMC.GenerateValues(1,1500,6000)
ImgMC.Update()

#Remplacer les polyDataNormals d'entrées avec les PolyData transformées
NormalsMC=vtk.vtkPolyDataNormals()
NormalsMC.SetInputConnection(ImgMC.GetOutputPort())
#Normals.SetFeatureAngle(60)
NormalsMC.Update()

stripperMC=vtk.vtkStripper()
stripperMC.SetInputConnection(NormalsMC.GetOutputPort())
stripperMC.Update()


mapperMC = vtk.vtkPolyDataMapper()
mapperMC.SetInputConnection(stripperMC.GetOutputPort())
mapperMC.ScalarVisibilityOff()
actorMC = vtk.vtkActor()
actorMC.SetMapper(mapperMC)
actorMC.GetProperty().SetColor(0.7,1,0.5)

#Actor 2: RT Structures
# Lire la structure RT
#n=6 #Numero du RT Structure
RTStruc=vtkgdcm.vtkGDCMPolyDataReader()
RTStruc.SetFileName(CheminRTStructure)
RTStruc.Update()

#Utiliser la matrice des DICOM images pour les lier avec les vtk
Transf=vtk.vtkTransform()
Transf.SetMatrix(readerImg.GetPatientMatrix())
Transf.Inverse()
Transf.Update()

TransfFiltreN=vtk.vtkTransformPolyDataFilter()
TransfFiltreN.SetInputConnection(RTStruc.GetOutputPort(n))
TransfFiltreN.SetTransform(Transf)
TransfFiltreN.Update()

TransfFiltreO=vtk.vtkTransformPolyDataFilter()
TransfFiltreO.SetInputConnection(RTStruc.GetOutputPort(o))
TransfFiltreO.SetTransform(Transf)
TransfFiltreO.Update()


#Structure n
mapperRTStrucN = vtk.vtkPolyDataMapper()
mapperRTStrucN.SetInputConnection(TransfFiltreN.GetOutputPort())
#Structure o
mapperRTStrucO = vtk.vtkPolyDataMapper()
mapperRTStrucO.SetInputConnection(TransfFiltreO.GetOutputPort())

actorN = vtk.vtkActor()
actorN.SetMapper(mapperRTStrucN)
actorO = vtk.vtkActor()
actorO.SetMapper(mapperRTStrucO)


#Actor 3: DICOM images par coupes
# Lire les DICOM images par coupes

actorImg1 = vtk.vtkImageActor()#Afficher la 1ère coupe au choix
actorImg1.SetInputData(readerImg.GetOutput())
actorImg2 = vtk.vtkImageActor()#Afficher la 2ème coupe au choix
actorImg2.SetInputData(readerImg.GetOutput())
actorImg3 = vtk.vtkImageActor()#Afficher la 3ème coupe au choix
actorImg3.SetInputData(readerImg.GetOutput())

mapperImg1 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg1.GetMapper())
mapperImg2 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg2.GetMapper())
mapperImg3 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg3.GetMapper())
#mapperImg.SetSliceNumber(dim[2]/2)
actorImg1.GetMapper()
actorImg2.GetMapper()
actorImg3.GetMapper()


mapperImg1.SetSliceNumber(0)
mapperImg2.SetSliceNumber(30)
mapperImg3.SetSliceNumber(dim[2])

# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)

# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Ajouter actor dans le renderer

#Les images DICOM par coupes
ren.AddActor(actorImg1)
ren.AddActor(actorImg2)
ren.AddActor(actorImg3)
#Les images DICOM Marching cubes
#ren.AddActor(actorMC)
#Les structures
ren.AddActor(actorN)
ren.AddActor(actorO)


#ren.GetActiveCamera().Zoom(1.5)

# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()


In [3]:
######################################################################
#Chapitre 3: Importer les VTK et DICOM Images et les visualiser#
######################################################################


#Example Cochon TheraneaM

# Lire les DICOM images (version VTK)      NB: par défaut, critère du image number

PathDicom2="/home/jonathan/Bureau/DICOMcochon/UH"

directory = vtkDICOMPython.vtkDICOMDirectory()
directory.SetDirectoryName(PathDicom2)
directory.Update()

readerImg = vtkDICOMPython.vtkDICOMReader()
readerImg.SetFileNames(directory.GetFileNamesForSeries(0))
readerImg.Update()

ImgMC = vtk.vtkMarchingCubes()
ImgMC.SetInputConnection(readerImg.GetOutputPort())
ImgMC.GenerateValues(1,150,300)
ImgMC.Update()

dim = readerImg.GetOutput().GetDimensions()

#Remplacer les polyDataNormals d'entrées avec les PolyData transformées
NormalsMC=vtk.vtkPolyDataNormals()
NormalsMC.SetInputConnection(ImgMC.GetOutputPort())
#Normals.SetFeatureAngle(60)
NormalsMC.Update()

stripperMC=vtk.vtkStripper()
stripperMC.SetInputConnection(NormalsMC.GetOutputPort())
stripperMC.Update()

mapperMC = vtk.vtkPolyDataMapper()
mapperMC.SetInputConnection(stripperMC.GetOutputPort())
mapperMC.ScalarVisibilityOff()

actorMC = vtk.vtkActor()
actorMC.SetMapper(mapperMC)
actorMC.GetProperty().SetColor(1,1,0.5)


##########################################

# Lire les structures VTK

#1: Os
filenameOs="/home/jonathan/Bureau/DICOMcochon/vtk/Francisco.vtk"
VolumeOs = vtk.vtkPolyDataReader()
VolumeOs.SetFileName(filenameOs)
VolumeOs.Update()

#2: HolmiumG
filenameHoG="/home/jonathan/Bureau/DICOMcochon/vtk/HolmiumG.vtk"
VolumeHoG = vtk.vtkPolyDataReader()
VolumeHoG.SetFileName(filenameHoG)
VolumeHoG.Update()

#3: HolmiumD
filenameHoD="/home/jonathan/Bureau/DICOMcochon/vtk/HolmiumD.vtk"
VolumeHoD = vtk.vtkPolyDataReader()
VolumeHoD.SetFileName(filenameHoD)
VolumeHoD.Update()

################################################################
#Utiliser la matrice des DICOM images pour les lier avec les vtk

#Premièrement !
Transf1=vtk.vtkTransform()
Transf1.Scale(-1.,-1.,1.)
Transf1.Update()

TransfFiltreOs=vtk.vtkTransformPolyDataFilter()
TransfFiltreOs.SetInputConnection(VolumeOs.GetOutputPort())
TransfFiltreOs.SetTransform(Transf1)
TransfFiltreOs.Update()
TransfFiltreHoG=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG.SetInputConnection(VolumeHoG.GetOutputPort())
TransfFiltreHoG.SetTransform(Transf1)
TransfFiltreHoG.Update()
TransfFiltreHoD=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoD.SetInputConnection(VolumeHoD.GetOutputPort())
TransfFiltreHoD.SetTransform(Transf1)
TransfFiltreHoD.Update()

Transf=vtk.vtkTransform()
Transf.SetMatrix(readerImg.GetPatientMatrix())
Transf.Inverse()
Transf.Update()

TransfFiltreOs2=vtk.vtkTransformPolyDataFilter()
TransfFiltreOs2.SetInputConnection(TransfFiltreOs.GetOutputPort())
TransfFiltreOs2.SetTransform(Transf)
TransfFiltreOs2.Update()
TransfFiltreHoG2=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG2.SetInputConnection(TransfFiltreHoG.GetOutputPort())
TransfFiltreHoG2.SetTransform(Transf)
TransfFiltreHoG2.Update()
TransfFiltreHoD2=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoD2.SetInputConnection(TransfFiltreHoD.GetOutputPort())
TransfFiltreHoD2.SetTransform(Transf)
TransfFiltreHoD2.Update()

#Deuxièmement !
Transf2=vtk.vtkTransform()
Transf2.Scale(1.,1.,-1.)
Transf2.Update()

TransfFiltreOs3=vtk.vtkTransformPolyDataFilter()
TransfFiltreOs3.SetInputConnection(TransfFiltreOs2.GetOutputPort())
TransfFiltreOs3.SetTransform(Transf2)
TransfFiltreOs3.Update()
TransfFiltreHoG3=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG3.SetInputConnection(TransfFiltreHoG2.GetOutputPort())
TransfFiltreHoG3.SetTransform(Transf2)
TransfFiltreHoG3.Update()
TransfFiltreHoD3=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoD3.SetInputConnection(TransfFiltreHoD2.GetOutputPort())
TransfFiltreHoD3.SetTransform(Transf2)
TransfFiltreHoD3.Update()


#Activer cette section selon besoins

NormalsOs=vtk.vtkPolyDataNormals()
NormalsOs.SetInputConnection(TransfFiltreOs3.GetOutputPort())
NormalsOs.Update()
NormalsHoG=vtk.vtkPolyDataNormals()
NormalsHoG.SetInputConnection(TransfFiltreHoG3.GetOutputPort())
NormalsHoG.Update()
NormalsHoD=vtk.vtkPolyDataNormals()
NormalsHoD.SetInputConnection(TransfFiltreHoD3.GetOutputPort())
NormalsHoD.Update()

#1: Os
mapperOs = vtk.vtkPolyDataMapper()
mapperOs.SetInputConnection(NormalsOs.GetOutputPort())
mapperOs.ScalarVisibilityOff()
actorOs = vtk.vtkActor()
actorOs.SetMapper(mapperOs)
actorOs.GetProperty().SetColor(0.8,1,0.3)

#2: HoG
mapperHoG = vtk.vtkPolyDataMapper()
mapperHoG.SetInputConnection(NormalsHoG.GetOutputPort())
mapperHoG.ScalarVisibilityOff()
actorHoG = vtk.vtkActor()
actorHoG.SetMapper(mapperHoG)
actorHoG.GetProperty().SetColor(1,0,1)

#3: HoD
mapperHoD = vtk.vtkPolyDataMapper()
mapperHoD.SetInputConnection(NormalsHoD.GetOutputPort())
mapperHoD.ScalarVisibilityOff()
actorHoD = vtk.vtkActor()
actorHoD.SetMapper(mapperHoD)
actorHoD.GetProperty().SetColor(0.9,0.2,0.5)

#Actor 3: DICOM images par coupes
# Lire les DICOM images par coupes

actorImg1 = vtk.vtkImageActor()#Afficher la 1ère coupe au choix
actorImg1.SetInputData(readerImg.GetOutput())
actorImg2 = vtk.vtkImageActor()#Afficher la 2ème coupe au choix
actorImg2.SetInputData(readerImg.GetOutput())
actorImg3 = vtk.vtkImageActor()#Afficher la 3ème coupe au choix
actorImg3.SetInputData(readerImg.GetOutput())

mapperImg1 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg1.GetMapper())
mapperImg2 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg2.GetMapper())
mapperImg3 = vtk.vtkImageSliceMapper().SafeDownCast(actorImg3.GetMapper())
actorImg1.GetMapper()
actorImg2.GetMapper()
actorImg3.GetMapper()
mapperImg1.SetSliceNumber(0)
mapperImg2.SetSliceNumber(dim[2]/5)
mapperImg3.SetSliceNumber(dim[2]/3)

# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)

# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Assign actor to the renderer
ren.AddActor(actorOs)
ren.AddActor(actorHoD)
ren.AddActor(actorHoG)
ren.AddActor(actorMC)
#Les images DICOM par coupes
ren.AddActor(actorImg1)
ren.AddActor(actorImg2)
ren.AddActor(actorImg3)

#ren.GetActiveCamera().Zoom(1.5)
# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()





In [2]:
################################################################################
#Chapitre 4: Utilisation du Image Data Stencil pour Holmium isolé (version VTK)#
################################################################################


#Volume: HolmiumG
filenameHoG="/home/jonathan/Bureau/DICOMcochon/vtk/HolmiumG.vtk"
VolumeHoG = vtk.vtkPolyDataReader()
VolumeHoG.SetFileName(filenameHoG)
VolumeHoG.Update()

# Lire les DICOM images (version VTK)      NB: par défaut, critère du image number

PathDicom2="/home/jonathan/Bureau/DICOMcochon/UH"

directory = vtkDICOMPython.vtkDICOMDirectory()
directory.SetDirectoryName(PathDicom2)
directory.Update()

readerImg = vtkDICOMPython.vtkDICOMReader()
readerImg.SetFileNames(directory.GetFileNamesForSeries(0))
readerImg.Update()

################################################################
#Utiliser la matrice des DICOM images pour les lier avec les vtk

#Premièrement !
Transf1=vtk.vtkTransform()
Transf1.Scale(-1.,-1.,1.)
Transf1.Update()
TransfFiltreHoG=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG.SetInputConnection(VolumeHoG.GetOutputPort())
TransfFiltreHoG.SetTransform(Transf1)
TransfFiltreHoG.Update()
Transf=vtk.vtkTransform()
Transf.SetMatrix(readerImg.GetPatientMatrix())
Transf.Inverse()
Transf.Update()
TransfFiltreHoG2=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG2.SetInputConnection(TransfFiltreHoG.GetOutputPort())
TransfFiltreHoG2.SetTransform(Transf)
TransfFiltreHoG2.Update()
#Deuxièmement !
Transf2=vtk.vtkTransform()
Transf2.Scale(1.,1.,-1.)
Transf2.Update()
TransfFiltreHoG3=vtk.vtkTransformPolyDataFilter()
TransfFiltreHoG3.SetInputConnection(TransfFiltreHoG2.GetOutputPort())
TransfFiltreHoG3.SetTransform(Transf2)
TransfFiltreHoG3.Update()
NormalsHoG=vtk.vtkPolyDataNormals()
NormalsHoG.SetInputConnection(TransfFiltreHoG3.GetOutputPort())
NormalsHoG.Update()


#Etape du stencil
dataToStencil = vtk.vtkPolyDataToImageStencil()
dataToStencil.SetInputConnection(NormalsHoG.GetOutputPort())
dataToStencil.SetOutputSpacing(readerImg.GetOutput().GetSpacing())
#dataToStencil.SetOutputOrigin(Origine)
stencil = vtk.vtkImageStencil()
stencil.SetInputConnection(readerImg.GetOutputPort())
stencil.SetStencilConnection(dataToStencil.GetOutputPort())
#stencil.ReverseStencilOn()
stencil.SetBackgroundValue(0)
stencil.Update()



#Visualisation
imageAppend=vtk.vtkImageAppend()
imageAppend.AddInputConnection(stencil.GetOutputPort())
viewer = vtk.vtkImageViewer()
interator = vtk.vtkRenderWindowInteractor()
viewer.SetInputConnection(imageAppend.GetOutputPort())
viewer.SetupInteractor(interator)
viewer.SetZSlice(130)
viewer.SetSize(1024,1024)




viewer.Render()
interator.Start()



In [ ]:
###################################
#Annexe 1: Visualiseur de stl seul#
###################################

filename = "/home/jonathan/Bureau/DICOM/stl/MetFrontG.stl"
 
reader = vtk.vtkSTLReader()
reader.SetFileName(filename)
reader.Update()

transform=vtk.vtkTransform()
#transform.Scale(-1,-1,1)

transformFilter=vtk.vtkTransformPolyDataFilter()
transformFilter.SetInputConnection(reader.GetOutputPort())
transformFilter.SetTransform(transform)
transformFilter.Update()

print(transformFilter.GetOutput().GetBounds())
    
mapper = vtk.vtkPolyDataMapper()
if vtk.VTK_MAJOR_VERSION <= 5:
    mapper.SetInput(reader.GetOutput())
else:
    mapper.SetInputConnection(reader.GetOutputPort())

actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)
# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# Assign actor to the renderer
ren.AddActor(actor)

# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()


In [3]:
############################################
#Annexe 2: Visualiseur de RTStructure seule#
############################################

n=6 #Numero du RT Structure
RTStruc=vtkgdcm.vtkGDCMPolyDataReader()
RTStruc.SetFileName("/home/jonathan/Bureau/DICOM/RTStruc/RTST0173.dcm")
RTStruc.Update()



mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(RTStruc.GetOutputPort(n))
actor = vtk.vtkActor()
actor.SetMapper(mapper)
 
# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)

# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# Assign actor to the renderer
ren.AddActor(actor)
 
# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()


In [ ]:
###################################
#Annexe 3: Visualiseur de vtk seul#
##################################

filename="/home/jonathan/Bureau/DICOMcochon/vtk/Segment_Os.vtk"
RTStruc = vtk.vtkPolyDataReader()
RTStruc.SetFileName(filename)
RTStruc.Update()

mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(RTStruc.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)
 
# Create a rendering window and renderer
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetSize(1024, 1024)

# Create a renderwindowinteractor
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
# Assign actor to the renderer
ren.AddActor(actor)
 
# Enable user interface interactor
iren.Initialize()
renWin.Render()
iren.Start()
